In [1]:
import pandas  as pd
import numpy as np
import os
%matplotlib inline

In [4]:
#!cp ../AnalyzeReferences/ng.3805-S2.xlsx .

In [20]:

MHB_df=pd.read_excel('ng.3805-S2.xlsx',sheetname='S. Table 1')

MHB_df['name']=MHB_df.index

windowSize=100
MHB_df['start']=MHB_df.start-windowSize
MHB_df['end']=MHB_df.end+windowSize

In [21]:
(MHB_df.start<MHB_df.end).mean()

1.0

In [27]:
MHB_df.to_csv('./MHB.bed',header=None,index=None,sep='\t')

In [46]:
!bedtools getfasta -fi /cellar/users/btsui/Data/Project/KangZhang/refFa/hg19.fa -tab -bed ./MHB.bed >MHB.tsv

### bisulfite convert the MHB.fa

In [50]:
!head MHB.tsv

chr1:237687-237918	GACCCACAGATGTGAAGCTGAGGCTGAAGGAGACTGATGTGGTTTCTCCTCAGTTTCTCTGTGCAGCACCAGGTGGCAGCAGAGGTCAGCAAGGCAAACCCGAGCCCGGGGATGCGGAGTGGGGGCAGCTACGTCCTCTCTTGAGCTACAGCAGATTCACTCTGTTCTGTTTCATTGTTGTTTAGTTTGCGTTGTGTTTCTCCAACTTTGTGCCTCATCAGGAAAAGCTTT
chr1:564370-564633	ATACCTAAACTAAGGTCTATGAACAATAAGATGATTTTCTTCAGTGGGACTTTTTTGTTTAATATATTAGATTTGACCTTCAGCAAGGTCAAAGGGAGTCCGAACTAGTCTCAGGCTTCAACATCGAATACGCCGCAGGCCCCTTCGCCCTATTCTTCATAGCCGAATACACAAACATTATTATAATAAACACCCTCACCACTACAATCTTCCTAGGAACAACATATAACGCACTCTCCCCTGAACTCTACACAACATATTTT
chr1:565685-566001	TCCAGCACCACAACCCTACTACTATCTCGCACCTGAAACAAGCTAACATGACTAACACCCTTAATTCCATCCACCCTCCTCTCCCTAGGAGGCCTGCCCCCGCTAACCGGCTTTTTGCCCAAATGGGCCATTATCGAAGAATTCACAAAAAACAATAGCCTCATCATCCCCACCATCATAGCCATCATCACCCTCCTTAACCTCTACTTCTACCTGCGCCTAATCTACTCCACCTCAATCACACTACTCCCTATATCTAACAACGTAAAAATAAAATGACAGTTTGAACACACAAAACCCACCCCATTCCTCCCCA
chr1:566622-566912	TCGTCACAGCCCATGCATTTGTAATAATCTTCTTCATAGTAATACCCATCATAATCGGAGGCTTTGGCAACTGACTAGTTCCCCTAATAATCGGTGCCCCCGATATGGCGT

In [2]:
MHB_seq_df=pd.read_csv('MHB.tsv',sep='\t',header=None)

In [3]:
MHB_seq_df.columns=['Region','Seq']

In [4]:
MHB_seq_df['Seq']=MHB_seq_df.Seq.str.upper()

In [5]:
#MHB_seq_df.Seq.str.count('CG').median()

In [6]:
cgProtectedS=MHB_seq_df.Seq.str.replace('CG','KG')
tmpS=cgProtectedS.str.replace('C','T') #convert unprotected C to become T
methylated_C_to_T_S=tmpS.str.replace('K','C')

In [7]:
MHB_seq_df['unmethylated_C_to_T_Seq']=MHB_seq_df.Seq.str.replace('C','T')

In [8]:
MHB_seq_df['methylated_C_to_T_Seq']=methylated_C_to_T_S

In [9]:
MHB_seq_df.head()

,Region,Seq,unmethylated_C_to_T_Seq,methylated_C_to_T_Seq
0,chr1:237687-237918,GACCCACAGATGTGAAGCTGAGGCTGAAGGAGACTGATGTGGTTTC...,GATTTATAGATGTGAAGTTGAGGTTGAAGGAGATTGATGTGGTTTT...,GATTTATAGATGTGAAGTTGAGGTTGAAGGAGATTGATGTGGTTTT...
1,chr1:564370-564633,ATACCTAAACTAAGGTCTATGAACAATAAGATGATTTTCTTCAGTG...,ATATTTAAATTAAGGTTTATGAATAATAAGATGATTTTTTTTAGTG...,ATATTTAAATTAAGGTTTATGAATAATAAGATGATTTTTTTTAGTG...
2,chr1:565685-566001,TCCAGCACCACAACCCTACTACTATCTCGCACCTGAAACAAGCTAA...,TTTAGTATTATAATTTTATTATTATTTTGTATTTGAAATAAGTTAA...,TTTAGTATTATAATTTTATTATTATTTCGTATTTGAAATAAGTTAA...
3,chr1:566622-566912,TCGTCACAGCCCATGCATTTGTAATAATCTTCTTCATAGTAATACC...,TTGTTATAGTTTATGTATTTGTAATAATTTTTTTTATAGTAATATT...,TCGTTATAGTTTATGTATTTGTAATAATTTTTTTTATAGTAATATT...
4,chr1:566894-567212,TTCTCCTTACACCTAGCAGGTATCTCCTCTATCTTAGGAGCCATCA...,TTTTTTTTATATTTAGTAGGTATTTTTTTTATTTTAGGAGTTATTA...,TTTTTTTTATATTTAGTAGGTATTTTTTTTATTTTAGGAGTTATTA...


In [18]:
#subDf=MHB_seq_df[['Region','unmethylated_C_to_T_Seq']].copy()#.stack()
#subDf['Name']=''
myFastaS=MHB_seq_df.apply(lambda S: '>'+S.loc['Region']+'_unmethylated'+'\n'+S.loc['unmethylated_C_to_T_Seq']+'\n'
            +'>'+S.loc['Region']+'_methylated'+'\n'+S.loc['methylated_C_to_T_Seq']+'\n',
            axis=1)

In [19]:
#myRef=myFastaS.sum()

In [20]:
genomePrepFolder='/cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/'


In [21]:
len(myFastaS)

147888

In [38]:
with open(genomePrepFolder+'MHB.fa','w') as f:
    for i,l in enumerate(myFastaS):
        f.write(l)


In [39]:
#!head /cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/MHB.fa

In [40]:
rapmapCmdFmt='/cellar/users/btsui/Program/rapmap/RapMap-master/bin/rapmap quasiindex --noClip -t {ref} -i {ref_out}'
myCommand=rapmapCmdFmt.format(ref=genomePrepFolder+'MHB.fa'
                             ,ref_out='/cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/')
myCommand

'/cellar/users/btsui/Program/rapmap/RapMap-master/bin/rapmap quasiindex --noClip -t /cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/MHB.fa -i /cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/'

In [41]:
!/cellar/users/btsui/Program/rapmap/RapMap-master/bin/rapmap quasiindex --noClip -t /cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/MHB.fa -i /cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/

RapMap Indexer

[Step 1 of 4] : counting k-mers
counted k-mers for 290000 transcriptsElapsed time: 4.36501s

Replaced 2 non-ATCG nucleotides
Clipped poly-A tails from 0 transcripts
Building rank-select dictionary and saving to disk done
Elapsed time: 0.00422739s
Writing sequence data to file . . . done
Elapsed time: 0.0967445s
[info] Building 32-bit suffix array (length of generalized text is 87592964)
Building suffix array . . . success
saving to disk . . . done
Elapsed time: 0.12098s
done
Elapsed time: 6.07712s
processed 87000000 positions
khash had 49515175 keys
saving hash to disk . . . done
Elapsed time: 2.64603s


In [30]:
#bismark_genome_preparation
baseBismarkDir='/cellar/users/btsui/Program/bismark/bismark_v0.14.3//'

In [35]:
#!mv MHB.fa 
#sort into bam. Check the count at the end for the reads mapped, does it generate something that resembl bimodal distribution?

In [32]:
#!cp MHB.fa  /cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/.

In [39]:
referencePrepCmd=baseBismarkDir+'bismark_genome_preparation '+genomePrepFolder

In [38]:
#!/cellar/users/btsui/Program/bismark/bismark_v0.14.3//bismark_genome_preparation /cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/

In [44]:
!head /cellar/users/btsui/Data/RAPMAP_INDEX/HMB/Homo_sapiens/Bisulfite_Genome/CT_conversion/genome_mfa.CT_conversion.fa

>chr1:237687-237918_CT_converted
GATTTATAGATGTGAAGTTGAGGTTGAAGGAGATTGATGTGGTTTTTTTTTAGTTTTTTTGTGTAGTATTAGGTGGTAGTAGAGGTTAGTAAGGTAAATTTGAGTTTGGGGATGTGGAGTGGGGGTAGTTATGTTTTTTTTTGAGTTATAGTAGATTTATTTTGTTTTGTTTTATTGTTGTTTAGTTTGTGTTGTGTTTTTTTAATTTTGTGTTTTATTAGGAAAAGTTTT
>chr1:564370-564633_CT_converted
ATATTTAAATTAAGGTTTATGAATAATAAGATGATTTTTTTTAGTGGGATTTTTTTGTTTAATATATTAGATTTGATTTTTAGTAAGGTTAAAGGGAGTTTGAATTAGTTTTAGGTTTTAATATTGAATATGTTGTAGGTTTTTTTGTTTTATTTTTTATAGTTGAATATATAAATATTATTATAATAAATATTTTTATTATTATAATTTTTTTAGGAATAATATATAATGTATTTTTTTTTGAATTTTATATAATATATTTT
>chr1:565685-566001_CT_converted
TTTAGTATTATAATTTTATTATTATTTTGTATTTGAAATAAGTTAATATGATTAATATTTTTAATTTTATTTATTTTTTTTTTTTTAGGAGGTTTGTTTTTGTTAATTGGTTTTTTGTTTAAATGGGTTATTATTGAAGAATTTATAAAAAATAATAGTTTTATTATTTTTATTATTATAGTTATTATTATTTTTTTTAATTTTTATTTTTATTTGTGTTTAATTTATTTTATTTTAATTATATTATTTTTTATATTTAATAATGTAAAAATAAAATGATAGTTTGAATATATAAAATTTATTTTATTTTTTTTTA
>chr1:566622-566912_CT_converted
TTGTTATAGTTTATGTATTTGTAATAATTTTTTTTATAGTAATATTTATTATAAT

In [44]:
!tail mapped_reads.sam

SRR019649.19839943	0	chr17:53951840-53952063_unmethylated	51	255	87M	*	0	0	TGTGTTTAAGTGTTTTTATTGTTTAATTTTTATTTATGAGTGAGAATAGGTAGTGTTTTGTTTTTTGTTTTTGTGATAGTTTGTTAG	*	NH:i:4
SRR019649.19839943	2304	chr17:53951840-53952063_methylated	51	255	87M	*	0	0	TGTGTTTAAGTGTTTTTATTGTTTAATTTTTATTTATGAGTGAGAATAGGTAGTGTTTTGTTTTTTGTTTTTGTGATAGTTTGTTAG	*	NH:i:4
SRR019649.19839943	2304	chr18:54583401-54583621_unmethylated	143	255	78M9S	*	0	0	TGTGTTTAAGTGTTTTTATTGTTTAATTTTTATTTATGAGTGAGAATAGGTAGTGTTTTGTTTTTTGTTTTTGTGATAGTTTGTTAG	*	NH:i:4
SRR019649.19839943	2304	chr18:54583401-54583621_methylated	143	255	78M9S	*	0	0	TGTGTTTAAGTGTTTTTATTGTTTAATTTTTATTTATGAGTGAGAATAGGTAGTGTTTTGTTTTTTGTTTTTGTGATAGTTTGTTAG	*	NH:i:4
SRR019649.19839944	0	chr10:77978838-77979193_unmethylated	280	255	76M11S	*	0	0	TAGGAGATTTATTTTATGTGTAGAGATATATATAGGTTTAAAATAAAGGGATGGAGAAAGATTTATTAAGTAAATGGAAAGATCGGA	*	NH:i:1
SRR019649.19839956	0	chr8:114845129-114845530_unmethylated	26	255	87M	*	0	0	TGAATTTTTGAAAAGATTAATAATAAGATTTGAAATTGAGGTAGTAATT